# Thinking 1 ：当我们思考数据源的时候，都有哪些维度，如果你想要使用爬虫抓取数据，都有哪些工具？
>答：数据源大概分为四类：1.公开开放的数据集，如《统计年鉴》等。  
2.可以爬取的网页数据，如，网站内容或者APP内容等，注意robots协议。  
3.传感器数据，如无人驾驶汽车雷达采集信息等，一般是物理信息。  
4.日志信息，用于统计用户的操作，可以前端埋点收集或者后端脚本收集及统计，以统计网站的访问情况及瓶颈等，如网站上一个商品的点击率数据。  

>爬虫工具有两类：一类为可视化工具，以八爪鱼和火车头为代表。特点速度快，模板多，但不全面不够个性化。  
另一类为Python爬虫工具，主要有BeautifulSoup和Selenium两个库，Selenium主要用于模拟人的操作。两者均可以自定义爬取内容。

# Thinking 2 ：企业的数据源来自多个维度，请思考什么是企业的数据中台，你是如何理解一方数据，二方数据，三方数据？
> 答：一方数据主要指自有的数据，以淘宝为例，一方数据为：各个商品的价格，描述，商家所在地等。  
二方数据为媒体交互行为数据，如广告监测，网站分析，舆情分析等，以淘宝为例，二方数据为：各个商品的点击度等。    
三方数据为第三方数据供应商提供的数据，如数据平台，运营商等，以淘宝商家为例，三方数据为：各个商家得到的淘宝提供的信息数据，此处淘宝为第三方数据供应商。  

# Action 1 : 不用任何数学库，如何求出sqrt(10)，并且精确到小数点后10位

In [26]:
# 二分法
def sqrt(num, error):
    max_ = num
    min_ = 0 
    mid = (max_ + min_)/2
    while (mid**2 - num >= error) or (num - mid**2 >= error):
        if mid**2 - num > 0:
            max_ = mid
            mid = (max_ + min_)/2
            
            continue
        else:
            min_ = mid
            mid = (max_ + min_)/2
            
            continue
    return mid    
print(sqrt(10,1e-10))

import numpy as np
print(np.sqrt(10))


3.162277660157997
3.1622776601683795


# Action 2: 汽车投诉信息采集：
数据源：http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-1.shtml  
投诉编号，投诉品牌，投诉车系，投诉车型，问题简述，典型问题，投诉时间，投诉状态  
可以采用Python爬虫，或者第三方可视化工具  

In [160]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests

url = 'http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-1.shtml'
headers={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36'}
html = requests.get(url, headers = headers, timeout = 10)
#得到网页的源码
content = html.text

#先解析网页
soup =  bs(content, 'html.parser', from_encoding='utf-8')
table = soup.find('div', class_ ='tslb_b')
tr_list = table.find_all('tr')

#得到Feature
feature = tr_list[0]
feature_list = feature.find_all('th')
feature_list = [i.text for i in feature_list]
feature_list

#得到投诉列表
td_list = [i.find_all('td') for i in tr_list]
len(td_list)
table = []
for i in range(1,len(td_list)):
    td = [j.text for j in td_list[i]]
    table.append(td)
    
#得到最后的列表    
data_result = pd.DataFrame(table, columns = feature_list)
data_result

C:\Anaconda\lib\site-packages\bs4\__init__.py:220: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


,投诉编号,投诉品牌,投诉车系,投诉车型,问题简述,典型问题,投诉时间,投诉状态
0,513784,东风小康,风光330,2018款 1.5L 330S 精典型 国V DK15,东风风光330行驶中右后车轮半轴损坏,"G204,",2020-08-25,信息审核
1,513782,一汽-大众奥迪,奥迪A4L,2016款 2.0T CVT 自标准型 典藏版,一汽大众奥迪A4L发动机节温器漏防冻液,"A37,",2020-08-25,信息审核
2,513779,东风标致,标致408,2014款 1.6T 自动 尊贵版,东风标致408电瓶亏电 4S店不解决问题,"H146,I295,",2020-08-25,信息审核
3,513775,广汽本田,冠道,2019款 240TURBO 两驱 舒享版,广汽本田冠道后车窗密封条出现断裂,"H48,",2020-08-25,信息审核
4,513772,上汽通用五菱,宝骏630,2011款 1.5L 手动 标准型,宝骏630漆面鼓包希望厂家处理,"H81,",2020-08-25,信息审核
5,513755,东风悦达起亚,起亚K3,2016款 1.6L 自动 GLS,东风悦达起亚K3打方向时存在严重异响,"D59,",2020-08-25,信息审核
6,513749,一汽-大众奥迪,奥迪A4L,2020款 40 TFSI 时尚动感型,一汽大众奥迪A4L车窗升降异响 热车怠速抖动,"A9,E42,H72,",2020-08-25,信息审核
7,513748,一汽-大众,速腾,2019款 280TSI DSG 舒适型 国VI,一汽大众速腾新车行驶中变速箱异响,"B33,",2020-08-25,信息审核
8,513746,吉利汽车,帝豪GL,2017款 1.3T 手动 精英型,吉利帝豪GL正常行驶中离合器失效影响使用,"C267,J298,J301,",2020-08-25,信息审核
9,513745,江淮汽车,瑞风S3,2016款 1.5L 手动 豪华智能型,江淮瑞风S3前胎吃胎严重存在质量问题\t,"G245,",2020-08-25,信息审核


# 上述方法不显示具体问题内容，进行改进

In [203]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests

url = 'http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-1.shtml'
headers={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36'}
html = requests.get(url, headers = headers, timeout = 10)
#得到网页的源码
content = html.text

#先解析网页
soup =  bs(content, 'html.parser', from_encoding='utf-8')
table = soup.find('div', class_ ='tslb_b')
tr_list = table.find_all('tr')

#得到Feature
feature = tr_list[0]
feature_list = feature.find_all('th')
feature_list = [i.text for i in feature_list]
feature_list

#得到投诉列表
td_list = [i.find_all('td') for i in tr_list]
len(td_list)
table = []
for i in range(1,len(td_list)):
    td = [j.text for j in td_list[i]]
    table.append(td)
    
#得到最后的列表    
data_result_origin = pd.DataFrame(table, columns = feature_list)
# print(data_result_origin)


#收集典型问题数据
#原因：html解析文件中不包含此信息 ==》》 使用selenium
#典型问题开头字母有明确含义，需要进行匹配
import re
import numpy as np
problem_list = []
for i in data_result_origin['典型问题']:
    problem_codename = re.findall(r'[A-Z]+', i)
    problem_list.append(problem_codename)
print(problem_list)
print(len(problem_list))

data_result = pd.DataFrame()
data_result['服务问题数量'] = [i.count('I')+i.count('J')+i.count('K')+ i.count('L')+i.count('M')+i.count('N')+i.count('O')+i.count('P') for i in problem_list]
data_result['其他问题数量'] = [i.count('Q') for i in problem_list]
data_result['基本问题数量'] = [(len(i) - i.count('Q') - (i.count('I')+i.count('J')+i.count('K') +i.count('L')+i.count('M')+i.count('N')+i.count('O')+i.count('P'))) for i in problem_list]
data_result['基本问题'] = None
data_result['服务问题'] = None
data_result['其他问题'] = None
data_result


#使用 selenium 得到问题列表
import json
import requests
from lxml import etree
import time
from selenium import webdriver

chrome_driver = 'C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe'  #chromedriver的文件位置
# 需要将chromedriver放到Chrome\Application目录下
driver = webdriver.Chrome(executable_path = chrome_driver)
url = 'http://www.12365auto.com/zlts/0-0-0-0-0-0_0-0-1.shtml'
# 去空格，去换行\n
def format_str(str):
    return str.replace('\n', '').replace(' ', '')

# 对页面进行抓取分析
#使用xpath Hit ctrl+shift+x ,调取信息（注意：去掉li-over）
driver.get(url)
time.sleep(1)
html = driver.find_element_by_xpath("//*").get_attribute("outerHTML")
html = etree.HTML(html)

#设置需要抓取的字段
#统计基本问题
basic_problem = {}
problem = {}
for i in range(30):
    if data_result['基本问题数量'][i] == 1:
        a = i+2
        problem[i] = html.xpath("/html/body/div[@id='content']/div[@class='com_sec']/div[@class='tslb_b']/table[@class='ar_c ar_c1']/tbody/tr["+str(a)+"]/td[@class='tsgztj']/span[@class='bw']/a") +  html.xpath("/html/body/div[@id='content']/div[@class='com_sec']/div[@class='tslb_b']/table[@class='ar_c ar_c1']/tbody/tr["+str(a)+"]/td[@class='tsgztj']/span[@class='wt']/a") 
        basic_problem[i] = tuple(j.text for j in problem[i])
        data_result['基本问题'][i] = basic_problem[i]

    if data_result['基本问题数量'][i] >= 1:
        basic_problem[i] = []
        for j in range(data_result['基本问题数量'][i]):
            problem[i] = html.xpath("/html/body/div[@id='content']/div[@class='com_sec']/div[@class='tslb_b']/table[@class='ar_c ar_c1']/tbody/tr["+str(i + 2)+"]/td[@class='tsgztj']/span[@class='bw']["+str(j+1)+"]/a") +  html.xpath("/html/body/div[@id='content']/div[@class='com_sec']/div[@class='tslb_b']/table[@class='ar_c ar_c1']/tbody/tr["+str(i + 2)+"]/td[@class='tsgztj']/span[@class='wt']["+str(j+1)+"]/a")
            basic_problem[i].append([g.text for g in problem[i]])
        data_result['基本问题'][i] = basic_problem[i]

basic_problem = {}
problem = {}
for i in range(30):        
    if data_result['服务问题数量'][i] == 1:
        a = i+2
        problem[i] = html.xpath("/html/body/div[@id='content']/div[@class='com_sec']/div[@class='tslb_b']/table[@class='ar_c ar_c1']/tbody/tr["+str(a)+"]/td[@class='tsgztj']/span[@class='fwwt']/a") +  html.xpath("/html/body/div[@id='content']/div[@class='com_sec']/div[@class='tslb_b']/table[@class='ar_c ar_c1']/tbody/tr["+str(a)+"]/td[@class='tsgztj']/span[@class='fw']/a") 
        basic_problem[i] = [j.text for j in problem[i]]
        data_result['服务问题'][i] = basic_problem[i]

    if data_result['服务问题数量'][i] >= 1:
        basic_problem[i] = []
        for j in range(data_result['服务问题数量'][i]):
            problem[i] = html.xpath("/html/body/div[@id='content']/div[@class='com_sec']/div[@class='tslb_b']/table[@class='ar_c ar_c1']/tbody/tr["+str(i + 2)+"]/td[@class='tsgztj']/span[@class='fwwt']["+str(j+1)+"]/a") +  html.xpath("/html/body/div[@id='content']/div[@class='com_sec']/div[@class='tslb_b']/table[@class='ar_c ar_c1']/tbody/tr["+str(i + 2)+"]/td[@class='tsgztj']/span[@class='fw']["+str(j+1)+"]/a")
            basic_problem[i].append([g.text for g in problem[i]])
        data_result['服务问题'][i] = basic_problem[i]

basic_problem = {}
problem = {}
for i in range(30):        
    if data_result['其他问题数量'][i] == 1:
        a = i+2
        problem[i] = html.xpath("/html/body/div[@id='content']/div[@class='com_sec']/div[@class='tslb_b']/table[@class='ar_c ar_c1']/tbody/tr["+str(a)+"]/td[@class='tsgztj']/span[@class='qtwt']/a") +  html.xpath("/html/body/div[@id='content']/div[@class='com_sec']/div[@class='tslb_b']/table[@class='ar_c ar_c1']/tbody/tr["+str(a)+"]/td[@class='tsgztj']/span[@class='qt']/a") 
        basic_problem[i] = [j.text for j in problem[i]]
        data_result['其他问题'][i] = basic_problem[i]

    if data_result['其他问题数量'][i] >= 1:
        basic_problem[i] = []
        for j in range(data_result['其他问题数量'][i]):
            problem[i] = html.xpath("/html/body/div[@id='content']/div[@class='com_sec']/div[@class='tslb_b']/table[@class='ar_c ar_c1']/tbody/tr["+str(i + 2)+"]/td[@class='tsgztj']/span[@class='qtwt']["+str(j+1)+"]/a") +  html.xpath("/html/body/div[@id='content']/div[@class='com_sec']/div[@class='tslb_b']/table[@class='ar_c ar_c1']/tbody/tr["+str(i + 2)+"]/td[@class='tsgztj']/span[@class='qt']["+str(j+1)+"]/a")
            basic_problem[i].append([g.text for g in problem[i]])
        data_result['其他问题'][i] = basic_problem[i]

#拼接得到最后的数据框        
results = data_result_origin.join(data_result.iloc[:, 3:], how = 'inner')
results

C:\Anaconda\lib\site-packages\bs4\__init__.py:220: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


[['H'], ['H', 'H', 'H', 'H'], ['F'], ['A'], ['B', 'I'], ['Q'], ['B', 'H'], ['D'], ['K'], ['E', 'E'], ['B', 'C'], ['A', 'B', 'D', 'G', 'H'], ['A', 'I'], ['F'], ['H', 'H', 'O'], ['A', 'H'], ['I', 'M'], ['B', 'E', 'G'], ['H'], ['H', 'I'], ['H'], ['A', 'A', 'A'], ['K', 'O'], ['B', 'I'], ['H', 'J', 'L'], ['H'], ['B', 'H'], ['G'], ['A'], ['H', 'I']]
30


C:\Anaconda\lib\site-packages\ipykernel_launcher.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

,投诉编号,投诉品牌,投诉车系,投诉车型,问题简述,典型问题,投诉时间,投诉状态,基本问题,服务问题,其他问题
0,513846,上汽大众,朗逸,2013款 1.6L 手动 风尚版,上汽大众朗逸车内顶棚塌陷严重,"H168,",2020-08-25,信息审核,"[[车身附件及电器, 部件开裂]]",None,None
1,513845,吉利汽车,帝豪GL,2017款 1.8L 手动 精英型,吉利帝豪GL下雨天前车门无法解锁 把手掉漆,"H81,H68,H103,H197,",2020-08-25,信息审核,"[[车身附件及电器, 漆面起泡开裂], [车身附件及电器, 后备箱故障], [车身附件及电器...",None,None
2,513842,一汽大众,捷达VS5,2019款 280TSI 自动 进取型,捷达VS5玲珑轮胎开裂存在质量问题,"F100,",2020-08-25,信息审核,"[[轮胎, 开裂]]",None,None
3,513841,北京现代,悦动,2018款 1.6L 自动 悦目版 GL 国VI,北京现代悦动发动机出现启动困难,"A44,",2020-08-25,信息审核,"[[发动机, 无法启动]]",None,None
4,513840,东风日产,逍客,2017款 2.0L CVT 精英版 国V,东风日产逍客行驶中变速箱顿挫4S店不予保修,"B19,I297,",2020-08-25,信息审核,"[[变速器, 顿挫]]","[[服务态度, 不予索赔]]",None
5,513838,广汽讴歌,讴歌CDX,2018款 2.0L 两驱 创享版 Hybrid,广汽讴歌CDX加速时动力电池异味要求解决,"Q322,",2020-08-25,信息审核,None,None,"[[其他, 疑似设计缺陷]]"
6,513835,广汽丰田,汉兰达,2018款 2.0T 四驱 豪华版 7座 国V,广汽丰田汉兰达挡把掉皮 挂挡时挡杆处异响,"B184,H179,",2020-08-25,信息审核,"[[变速器, 挡把总成故障], [车身附件及电器, 车身装饰脱落]]",None,None
7,513831,北京现代,领动,2016款 1.6L 自动 智炫·精英型,北京现代领动跑高速时岀现方向卡滞\t,"D123,",2020-08-25,信息审核,"[[转向系统, 卡滞]]",None,None
8,513830,东风标致,标致4008,2019款 400THP 尝先版,东风标致4008续保押金到期不退还,"K309,",2020-08-25,用户评分,None,"[[服务收费, 定（订）金纠纷]]",None
9,513824,东风日产,奇骏,2014款 2.0L XL CVT 两驱 舒适版,东风日产奇骏行驶中刹车过硬刹车失灵,"E29,E333,",2020-08-25,信息审核,"[[制动系统, 刹车失灵], [制动系统, 刹车变硬]]",None,None


In [210]:
results.to_csv('车质网.csv', encoding = 'UTF-8-sig')

# 数据的拼接
https://www.cnblogs.com/keye/p/10791705.html